Mining Social and Geographic Datasets
-----------------------------------

GEOG0115 Computer Lab Week 7 
-------------------------------

Note: Notebook might contain scripts and instructions adapted from GEOG0115, GEOG0051. 
Contributors: Stephen Law, Michal Iliev, Mateo Neira, Nikki Tanu, Thomas Keel, Gong Jie, Jason Tang and Demin Hu.

Overview of Content in this Jupyter Notebook
===============
> ### Lab Notebook 7.1: Machine Learning for travel mode prediction (tabular data)
> ### Lab Exercise 7.1: XGBoost (Gradient Boosting)

Lab Notebook 7.1: Machine Learning for travel mode prediction (tabular data)
-------------------------------

In this week's notebook, we will be reviewing some of the concepts and algorithms we learnt in previous three classes such as <b>logistic regression</b>, <b>decision tree</b>, <b>Random Forest</b> and <b>Gradient Boosting</b> for the travel mode classification problem. Think of this as a **review notebook** to put some of what you learn into practice.

## Machine Learning Application: Travel mode classification

An important task in urban analytics and transport planning is to predict the travel mode choice of individuals. The aim of the task is to predict which travel mode an individual takes namely `[car,bike,walk]`. This type of research can be conducted using  `trajectory` data but also travel survey data that is based on various socio-economic, climatic, individual and neighbourhood attributes such as `['distance', 'density', 'age', 'male', 'ethnicity','education', 'income', 'cars', 'license', 'bicycles', 'weekend','diversity', 'green', 'temp', 'precip', 'wind']`. 

The primary data source for this study is the Dutch national travel survey (NTS) conducted from 2010 to 2012 based on individual travel diaries. The survey participants were asked to record every trip in six days randomly selected over a year to capture seasonal effects. Below is the description of the input variables. Please read **(Hagenauera et al. 2017)** as part of this lab seminar. 

**Trip**	<br>
`distance`	Total trip distance in km <br>
`weekend`	Trip is done at the weekend <br>
`mode`	Main travel mode (walk, bike, pt, car). pt refers to public transport. <br>
**Individual**	<br>
`age`	Age of participant in years <br>
`education`	Education of participant (lower, middle, higher)<br>
`ethnicity`	Ethnicity of participant (native, western, other)<br>
`license`	Participant owns a driver’s license (yes, no)<br>
`male`	Male participant (yes, no)<br>
**Household**	<br>
`bicycles`	Number of bicycles per household<br>
`cars`	Number of cars per household<br>
`income`	Net annual household income in 1,000€ (<20,≥ 20–40,≥ 40)<br>
**Build and natural environment**	<br>
`density`	Address density, aggregated over post codes, in 1,000addresses per km2<br>
`diversity`	Shannon diversity index of land use classes<br>
`green`	Proportion of green space per post code area in %<br>
**Weather**	<br>
`precip`	Daily precipitation sum in mm<br>
`temp`	Daily maximum temperature in °C<br>
`wind`	Daily average wind speed in m/s<br>



[1] Hagenauera,J., Helbich,M. (2017) Comparative study of machine learning classifiers for modelling travel mode choice. Expert Systems with Applications (78)

#### Import Scikit-learn
You may notice below that we import only certain portions of the ```sklearn``` package, and this is so as to conserve memory on your operating system through importing just what we will use. Remember also that what we are introduced to in these notebooks are but a drop in the ocean of possibilities of how you could implement Machine Learning principles with Python, and so, as your interests guide you, feel free to explore content and functions beyond what is set out here and in addition to some the links for further reading placed in these notebooks.

For more infomation, look at [the official website](https://scikit-learn.org/).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## 7.1.1 Supervised Learning - Logistic Regression

### Logistic Regression (Baseline Model)

Let's start with running a logistic regression as the baseline classifier. As a reminder, a <b>Logistic Regression model</b> is a variant of a generalised linear models where a <b>sigmoid</b>  function $s(x)=\frac{1}{1+e^{-f(x)}}$ is typically used in translating a standard linear regression into a binary classification problem $y_i\in{0,1}$ which can be extended into a multi-class classification problem $y_i\in{1,...,k}$. While linear regression models predict an output that is a continuous variable, logistic regression models are instead used for classification with a discrete categorical outcome (E.g. whether the flower is an Iris or rather an individual takes a bus to school). These models predict the <b> probability</b> of a certain event or categorisation happening: based on the variables defined by the user, each observation will be given a probability of between 0 and 1 on the output (event/class) in happening. Thereafter, the observations can be classified into 2 or more classes based on user-defined thresholds. 

Note: The logistic regression function in ```sklearn``` implements regularized logistic regression to give a predicted output of 2 or more classes. Read more about it [here](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression) and [here](https://mlu-explain.github.io/logistic-regression/).

In [ ]:
# let's read in the national travel survey data
dft=pd.read_csv('Travel_Mode_Classification/nts_data.csv')

In [ ]:
Xcols=['distance', 'density', 'age', 'male', 'ethnicity',
       'education', 'income', 'cars', 'license', 'bicycles', 'weekend',
       'diversity', 'green', 'temp', 'precip', 'wind']
ycols=['mode_main']


In [ ]:
#define the x-variables (input) and y-variable (outcome of classification)
X = dft[Xcols]
y = dft[ycols]

In [ ]:
X.head()

In [ ]:
y.head()

let's first look at the dependent variable (categorical) of interests using a barplot.

In [ ]:
pd.DataFrame(y.value_counts()).plot(kind='barh',color='black')

#### 🤨 TASK
what do you notice? are the different classes balance? and which is the most popular mode of transport and which is the less?

In [ ]:
???

let's now take a look at the independent variables which has a total of 230,607 samples, there seems to be both numeric and categorical variables in the dataframe.

In [ ]:
X.info()

## Describing the dataset
 
Before building a classifier, it is important to **understand** and **describe** the data contained within the imported data frames. For example, how is the data distributed? Are there any missing values?

For numeric variables
* descriptive statistics
* histograms

For categorical variables
* bar-plot
* counts

Beyond that, depending on your application, it is also common to look at how the variables correlate with each other using a correlation matrix.

let's check whether there are any missing data first.

In [ ]:
missing_count = X.isna().sum()
print("Count of missing values")
print(missing_count)

let's then describe the data statistically. What was the mean/median `temp` and `percip` for example in the dataset? 


In [ ]:
X.describe()

For **numeric variables**, you can visualise the distribution of the data using histograms.

In [ ]:
numeric_data=X.select_dtypes(include=[np.number])
numeric_data.hist(figsize=(10,10),color='black',grid=False)
plt.axis(False)
plt.show()

you might notice most of the variables distributions are uni-modal, some are symmetrical while a couple are highly skewed. Do the distribution makes sense? Based on our previous learning, `distance` (distance travel) is highly skewed meaning the majority of the trips are shorter in distance. 


let's then describe the categorical variable using barplots which counts the frequency for each class.

In [ ]:
categorical_data = X.select_dtypes(include=[object])
categorical_data.columns

In [ ]:
fig,ax=plt.subplots(2,3,figsize=(20,10))
categorical_data['male'].value_counts().plot(kind='bar',color='black',ax=ax[0,0])
categorical_data['ethnicity'].value_counts().plot(kind='bar',color='black',ax=ax[0,1])
categorical_data['education'].value_counts().plot(kind='bar',color='black',ax=ax[0,2])
categorical_data['income'].value_counts().plot(kind='bar',color='black',ax=ax[1,0])
categorical_data['license'].value_counts().plot(kind='bar',color='black',ax=ax[1,1])
categorical_data['weekend'].value_counts().plot(kind='bar',color='black',ax=ax[1,2])
plt.show()

you might notice `male`,`education`,`income` has a more balanced class than `ethnicity`,`license` and `weekend`

# Data preparation
Before we can use the data in our model we always need to transform it, process it, standardise it, clean it, etc. 

As a reminder, a couple of common data preparation procedures including;
1. coding categorical variables into numeric variables
2. transforming the numerous variables to be more symmetrically distributed (it is not an assumption of machine learning models but in practice it often helps in training)
3. standardise the numerous variables to make the features more comparable and to improve stability in training.

Please read more [here](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling)



Let's now transform the highly skewed numeric variables so that it is less skewed. This is not a necessary assumption for most machine learning algorithm but it helps in practice. It also reduces the influence of the extreme values. 

In [ ]:
from scipy.stats import skew

# you can calculate the skewness of the variable by using skew function in scipy.stats
numeric_data=X.select_dtypes(include=[np.number])

skewed = X[numeric_data.columns].apply(lambda x: skew(x.dropna().astype(float)))

# these are all the variables that are very positively skewed. 
rskewed = skewed[(skewed > 1)].index

# these are all the variables that are very negatively skewed. 
lskewed = skewed[(skewed < -1)].index
print (f'leftskewed:{lskewed} and right-skewed:{rskewed}')

You will see a number of variables are highly right-skewed so let's transform these ones specifically. For right-skewed variables, log-transform the variable `X[rskewed] = np.log1p(X[rskewed])`. If there were any left-skewed variables, you would log transform the reflected skewed variable `X[lskewed] = np.log1p(np.max(X[lskewed])-X[lskewed])`. 

In [ ]:
# before transformation plot
X[rskewed].hist(bins=20,figsize=(15,7), color='skyblue', xlabelsize=0, ylabelsize=0, grid=False, layout=(2,5))
plt.show()

# log-transforms the right skewed reflected variables of the dataset 
X[rskewed] = np.log1p(X[rskewed])

# now plot again the variables after their log transformation
X[rskewed].hist(bins=20,figsize=(15,7), color='orange', xlabelsize=0, ylabelsize=0, grid=False, layout=(2,5))
plt.show()

### reminder note on transformation

What do you notice different pre- and post-transformation for the variables above? For which variables does the log-transformation seem to make it less skewed?  

It is important to note, data transformations is common to make skewed variable less skewed but it must be applied very cautiously and it isn't a necessary condition for running regression models and you loses interpretability for the specific feature. It's crucial to examine the distribution of your data and consider the context of your analysis before deciding whether to log transform the numeric variables. Additionally, if you do choose to log transform, ensure that you handle zero values appropriately (e.g., by adding a small constant before transformation if zero values are present).

Be sure to play around with the many different arguments that allow you to customise the look of your multi-grid histogram plots with the `.hist()` (and all the other functions you come to interface with). 

Other more advance forms of transforms (`power transform`) are detailed [here](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.power_transform.html).

### Correlations/Covariance between variables

We can use a correlation matrix to understand the association between variables. Variables are almost always correlated. Ideally, we want our input variables to not be totally correlated with one another. Significant correlation between our independent variables undermines the robustness of our model and interpretability.  

In [ ]:
# create correlation matrix to understand the association between variables
import seaborn as sns
corr = X[numeric_data.columns].corr()
f, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(corr, center=0,cmap=plt.get_cmap('magma_r'),
            square=True, linewidths=.05, annot=True, vmin=-1, vmax=1,ax=ax) 
plt.show()

#### 🤨 TASK
What do you notice? Which variables are highly correlated with each other and which are not? 
Why do you think `density` and `green` have a strong negative correlation? 

In [ ]:
???

As we learnt in the previous classes, there are many different methods to explore the data. `Seaborn` is a popular library to further interrogate the data. More information can be found [here](https://seaborn.pydata.org/).

#### Standardise data
We need to standardise the data (again!) to make them comparable and more stable for training. Remember the units also become less interpretable.

In [ ]:
X[numeric_data.columns].head()

In [ ]:
# this line creates an instance of the Standard Scaler to transform the data
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

In [ ]:
# this fits and transforms the data
X[numeric_data.columns] = scaler.fit_transform(X[numeric_data.columns]) #we standardise all variables, including the log transformed ones

In [ ]:
X[numeric_data.columns].head()

let's then cast the categorical variables into numeric variables for analysis.

In [ ]:
X=pd.concat([X,pd.get_dummies(X['ethnicity'])],axis=1)
X=pd.concat([X,pd.get_dummies(X['male'])],axis=1)
X=pd.concat([X,pd.get_dummies(X['education'])],axis=1)
X=pd.concat([X,pd.get_dummies(X['income'])],axis=1)
X=pd.concat([X,pd.get_dummies(X['license'])],axis=1)
X=pd.concat([X,pd.get_dummies(X['weekend'])],axis=1)

#The pd.concat() operation with pd.get_dummies() adds new columns to X (dummy variables) 
#and 'X.columns=' ensures that those columns have the desired labels

X.columns=['distance', 'density', 'age', 'male', 'ethnicity', 'education',
       'income', 'cars', 'license', 'bicycles', 'weekend', 'diversity',
       'green', 'temp', 'precip', 'wind', 'native_eth', 'nonwestern_eth', 'western_eth',
       'no_male', 'yes_male', 'higher_edu', 'lower_edu', 'middle_edu', '20to40_inc', 'less20_inc', 'more40_inc',
       'no_license', 'yes_license', 'no_weekend', 'yes_weekend']

#it is redundant to keep both 'no_license' and 'yes_license' etc. so we select only specific variables
X=X[['distance', 'density', 'age', 'cars', 'bicycles', 'diversity',
       'green', 'temp', 'precip', 'wind', 'native_eth', 'nonwestern_eth', 'western_eth',
       'yes_male', 'higher_edu', 'lower_edu', 'middle_edu','20to40_inc', 'less20_inc', 'more40_inc',
       'yes_license', 'yes_weekend']]

Do the same for the target variable (y)

In [ ]:
from sklearn.preprocessing import LabelEncoder
Label = LabelEncoder() #easy labelling with LabelEncoder
y=pd.concat([y,pd.DataFrame(Label.fit_transform(y),columns=['labels'])],axis=1)
ylabels=y['labels']

### Splitting the dataset into training and test groups

As we mentioned in the last couple of classes, in machine learning applications, it is **important** to split the entire dataset at least two sets - the `_train` and `_test` sets to see how well it generalises - i.e., is it over-fitted to the dataset you made it using, such that it has little worth when used to predict outcomes on another dataset? In many cases, you might need to have three dataset namely `train/val/test` where the validation dataset in this case would be only used for the purposes of hyper-parameter tuning. 

In [ ]:
#splits the datasets (both x- and y-variables) into the training and test sets
(X_train, X_test, y_train, y_test) = train_test_split(X, ylabels, train_size=0.7, random_state=1)

In [ ]:
#creates a new logistic regression model
model = LogisticRegression()
#fits (/configures) this model using the training data
model.fit(X_train, y_train)

#then tries to make a prediction using the test dataset's x-variables
y_pred=model.predict(X_test)
y_pred[0:10]

#### 🤨 TASK
**Always sanity check** : do the results make sense? How do these predictions compare to the ground-truth? Plot the first 10 samples in y_test.

In [ ]:
???

Remember that the logistic regression estimates **probabilities** (in this case, which travel mode an individual takes), then classifies them into the mode they most likely have taken. To have an understanding of how accurate our model was, we can, instead of asking the model to give us the final classifications (travel mode), return the probabilities that it calculated.

In [ ]:
#predict probabilities of allocation of each categories
y_pred_prob=model.predict_proba(X_test)
pd.DataFrame(y_pred_prob,columns=[y.mode_main.unique()]).head(n=10)

### Reporting Results - Classificaton

Let's report the result with accuracy (the number of correctly predicted divided by its total).

In [ ]:
#reports the prediction accuracy of the testset
print ('the out of sample test accuracy is : '+ str(round(accuracy_score(y_test, y_pred),3)))

Let's now compare the results to the training set prediction. A significant difference between training and test accuracy (e.g., high training accuracy (below) but low test accuracy (above)) would indicate overfitting.

In [ ]:
#reports the prediction accuracy of the trainset
y_pred_train=model.predict(X_train) 
print ('the in sample test accuracy is : '+ str(round(accuracy_score(y_train, y_pred_train),3)))

The results are really similar meaning the model is likely not to be overfitting. 

As a reminder, for more details of the model, we can use the ```classification_report``` function in ```sklearn```. The report shows some key metrics through which we can judge the accuracy of the logistic regression model we have created: **accuracy,precision, recall and f1-score**. The calculation of precision and recall statistics entails using measures such as the rates of true and false negatives/positives (by comparing the predicted and observed) while F1 score is a composite measure that takes into consideration both the former two. In general these measures should be reported specially when the data is imbalanced (as here). 

More formally;

$Precision = TP/(TP+FP)$

$Recall = TP/(TP+FN)$

$F1 = 2(Precision*Recall)/(Precision+Recall)$

Read more about these statistics [here](https://muthu.co/understanding-the-classification-report-in-sklearn/) or look at documentation [here](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-report). 

```classification_report``` provides the results for each category as well as the macro averages (i.e. averages across the categories).


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

### Confusion Matrix
Let's also produce the <b>confusion matrix</b>, which shows exactly what proportion of each travel choice was rightly and wrongly classified. The x- and y-axes of the plot below shows, respectively, the travel mode that was predicted (```Predicted label```) vs the ground truth (```True label```) is. Those along the diagonals are rightly predicted and those along the off diagonals are wrongly predicted or confused.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_estimator(model, X_test, y_test, normalize='true')

#### 🤨 TASK
what do you notice? which travel mode is making the most errors? why do you think is the case here?

In [ ]:
???

### Model comparison - Ensemble Learners
Comparing models is a critical aspect of machine learning. What makes using the syntax of `sklearn` so convenient is that you only need to make a few adjustments to run a different model ie. `model.fit`. In this case we will learn about `Decision Tree` Classifiers and `Random Forest` Classifiers.

Let's take this opportunity to try out a different model as an example. In this case we will use the `RandomForestClassifier`. `RF` is an `ensemble learning` method which uses multiple simple predictors in order to make a better prediction. The hypothesis is that multipler learners will generate better results than a single learner. For tabular data, `ensemble learning` algorithms are even outperforming deep neural networks such as transformers (Shwartz-Ziv et al 2022).

In greater detail, **Random Forest Regressors/ Classifiers** (Breiman 2001), is based on fitting a multitude of [Decision Trees](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)[1], each on samples drawn with replacement (i.e., bootstrap sample) from the training set. The outcomes from multiple simple models across multiple observations are often i. the one that is most often predicted (majority voting) across multiple trees for classification or ii. the mean or average prediction across multiple trees for regression.

And lastly, another ensemble learning method is **Gradient Boosting Regressors/ Classifiers** (Friedman 2001), which combines many weak "learners" into a single strong learner in an iterative fashion where each regressor tries to improve the previous model. The one implemented in `sklearn` is called `GradientBoostingClassifier`. 

There are many more machine learning algorithms and this is only the tip of the ice-berg. We will explore one in today's exericise. The aim here isn't to describe every one of these but rather to show the spectrum of methods where the student can then go and explore.  

<img src="Random Forest Diagram (wikipedia-commons).png" alt="drawing" width="620" align='center' >
<center><b>Fig 1. Random Forest Diagram (wikipedia-commons).</b></center>


[1] It is important to give a simple description of Decision Tree here. DT is another supervised learning algorithm that learns simple decision rules inferred from the data features. As implied by the 'tree' in its name, the model can pass each observation that goes through it via multiple branches (or routes) of decisions to help it derive a final outcome for that given observation). [For a premier of Decision Trees in plain English, this](https://towardsdatascience.com/decision-trees-explained-3ec41632ceb6) is a good link to have a brief read at! 

Breiman, L. (2001). Random forests. Machine learning, 45(1), 5-32. 

Friedman, J.(2001). Greedy Function Approximation: A Gradient Boosting Machine, The Annals of Statistics, 29(5)

Shwartz-Ziv, R., & Armon, A. (2022). Tabular data: Deep learning is not all you need. Information Fusion, 81, 84-90.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_forest =  RandomForestClassifier() 
model_forest.fit(X_train, y_train)
y_pred=model_forest.predict(X_test)
print ('the out of sample test accuracy for a RandomForestClassifier is : '+ str(round(accuracy_score(y_test, y_pred),3)))

You can see that the calculation takes longer, but the accuracy is higher.

## Hyper-parameter tuning and Cross Validation

Many models in machine learning have hyper-parameters which are parameters that are not directly learnt within the estimators but instead needs to be tuned. Typical hyper-parameters include `max_depth`(The maximum depth of the tree) and `n_estimators`(The number of trees in the forest) for [RandomForestClassifiers](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). Let's set randomly here `max_depth`=20. Remember to read the documentation of the model to know what each of these hyper-parameters mean.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_forest2 =  RandomForestClassifier(max_depth=20) 
model_forest2.fit(X_train, y_train)
y_pred=model_forest2.predict(X_test)
print ('the out of sample test accuracy for a RandomForestClassifier is : '+ str(round(accuracy_score(y_test, y_pred),3)))

Are the results better or worst? how do we know whether these parameters yield the best results?

### KFold Cross Validation

As mentioned previously, when training a machine learning model that includes hyper-parameters. It is important to split the training data into a validation set. However, partitioning the training data into a train and validation set drastically reduce the number of samples which can be used for learning the model, while the results can depend on the particular validation set. A useful and common resampling procedure is  k-fold Cross validation, the training set is split into k smaller folds, where a model is trained on each folds of the training data and validated on the remaining parts of the data. This is repeated until all folds are exhausted. The results are calculated on an out-of-sample testset.

<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" alt="drawing" width="420" align="left">


### GridSearchCV

It is possible and recommended to search the hyper-parameter space for the best cross validation score using cross validation. A simple way to do this is called the `GridSearchCV` which exhaustively generates candidates from a grid of parameter values specified . To reduce the number of possible models to test, we restrict the search where 
`max_depth`:[10,40] and `n_estimators`:[100]. 
You can read more about it from [here](https://scikit-learn.org/stable/modules/grid_search.html)
and [here](https://medium.com/all-things-ai/in-depth-parameter-tuning-for-random-forest-d67bb7e920d) for an overview of what the main parameters are, and how you could tune them.  <br/>

**The process may take a while or doesn't work on your machine if doesn't have enough disk space. You are welcome to use [Google Colab online](https://colab.research.google.com/).**

In [ ]:
from sklearn.model_selection import GridSearchCV# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [10, 40],
    'n_estimators': [100]#, 200, 300]# restricted the grid search to reduce compute time 
}
# Create a based model
model = RandomForestClassifier()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 3 # number of folds for cross validation
                           , n_jobs = -1 # use all processors
                            , verbose = 2 # what to display
                          )

In [ ]:
grid_search.fit(X_train, y_train)

Please note, `GridSearchCV` is applied only on the training set here when you create three folds. Remember you can also just simply split the training set into a training and validation set.

In [ ]:
model = grid_search.best_estimator_
y_pred=model.predict(X_test)
print ('the out of sample test accuracy for a RandomForestClassifier is : '+ str(round(accuracy_score(y_test, y_pred),3)))

#### 🤨 TASK
Try grid searching for more hyper-parameters. 

In [ ]:
???

The classification accuracy will likely have improved and even if it hasn't improve, we can say that the results are robust to different settings of hyper-parameters. 
You might have also noticed, due to this process of hyper-parameter tuning, it takes a lot longer to process.

### Feature Importance

Due to the black-box nature of most machine learning algorithms, there are no easy way to interpret similar coefficients. However, there are **interpretable ML** approaches to measure feature importance for methods like `Random Forest (RF)`. A **model-agnostic** way to interpret a machine learning model such as `RF` is [permutation importance](https://scikit-learn.org/stable/modules/permutation_importance.html#permutation-importance).

**The process may take a while or doesn't work on your machine if it doesn't have enough disk space. You are welcome to use [Google Colab online](https://colab.research.google.com/).**

In [ ]:
from sklearn.inspection import permutation_importance
# if it doesn't work,you meight need to change n_jobs
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)
coef = pd.DataFrame(result.importances_mean,index=X.columns,columns=['importance'])
coef = coef.reset_index()
coef.columns=['features','importance']
coef['importance']=np.abs(coef['importance'])
coef=coef.sort_values(by='importance',ascending=False)
plt.figure(figsize=(6,6))

sns.barplot(x='importance',y='features',data=coef,)
plt.show()

Lab Exercise 7.1 XGBoost 
-------------------------------

For this week's exercise, we will have the student explore a popular **gradient boosting** [1] algorithm implementation called **XGBoost** (Extreme Gradient Boosting), which is a type of ensemble gradient boosting algorithm where weak learners (simple models) are built sequentially to correct the errors made by the preceding models. In simple terms, gradient boosting focuses on reducing the residual errors iteratively. `sklearn` has an implementation of the algorithm called `gradientboosting` but the `xgboost` version is much quicker. 

You can find the reference for the algorithm with this link [2] and the python library here [3]. The library can be easily installed with pip or conda. 

`!pip install xgboost`

[1] Friedman, J.(2001). Greedy Function Approximation: A Gradient Boosting Machine, The Annals of Statistics, 29(5)

[2] Chen, T., & Guestrin, C. (2016). Xgboost: A scalable tree boosting system. In Proceedings of the 22nd acm sigkdd international conference on knowledge discovery and data mining (pp. 785-794).

[3] xgboost. https://xgboost.readthedocs.io/en/stable/#



For this exercise, please run a similar **predictions** using the `XGBoost` library. Remember this is a multi-class classification problem and not a regression problem so please use the appropriate method. Remember to tune the hyper-parameters of the classifier. For `xgboost` an important parameter to tune is `eta/learning rate` which shrinks the contribution of each tree to help prevent overfitting and improve generalization. 
[link](https://xgboost.readthedocs.io/en/stable/parameter.html). 

In [ ]:
#!pip install xgboost
import xgboost as xgb

# Prepare the data for XGBoost using the DMatrix class, which optimizes memory usage and computation
dtrain = xgb.DMatrix(X_train, label=y_train)  
dtest = xgb.DMatrix(X_test, label=y_test)    

# Set up parameters for XGBoost model training
params = {
    'max_depth': 3,               # Maximum depth of a tree to control model complexity and prevent overfitting
    'eta': 0.1,                   # Learning rate (step size); try adjusting this value for better convergence
    'objective': 'multi:softmax', # Specifies the loss function for multi-class classification
    'num_class': 4,               # Number of unique classes in the target variable
}

num_boost_round = 20 # Number of boosting rounds (iterations); higher values can improve model performance but may increase training time

# Train the model using the training data and defined parameters
bst = xgb.train(params, dtrain, num_boost_round)

# Make predictions on the test set using the trained model
preds = bst.predict(dtest)

# Evaluate the model's accuracy using the actual labels and predictions
accuracy = accuracy_score(y_test, preds) 
print("Accuracy: {:.2%}".format(accuracy))  # Print accuracy as a percentage with 2 decimal places


#### 🤨 TASK

It might be a good time now to read the following article and try to replicate some of the experiments of the paper.

[1] Hagenauera,J.& Helbich,M. (2017) Comparative study of machine learning classifiers for modelling travel mode choice. Expert Systems with Applications (78)


In [ ]:
???